## basic_data
### 准备好您的数据以进行培训
此模块定义了Learner内部用于训练模型的基本DataBunch对象。 这是泛型类，可以采用任何类型的fastai Dataset或DataLoader。 您将在每个应用程序的数据模块中找到有用的功能，以便直接为您创建此DataBunch。

<b>class DataBunch</b>

`DataBunch(train_dl:DataLoader, valid_dl:DataLoader, fix_dl:DataLoader=None, test_dl:Optional[DataLoader]=None, device:device=None, dl_tfms:Optional[Collection[Callable]]=None, path:PathOrStr='.', collate_fn:Callable='data_collate', no_check:bool=False)`

将train_dl，valid_dl和test_dl绑定在数据对象中。

它还确保所有数据加载器都在设备上并在绘制批处理时应用于它们dl_tfms（如规范化）。 path在内部用于存储临时文件，collate_fn传递给pytorch Dataloader（替换那里的那个）来解释如何整理为批处理选择的样本。 默认情况下，它将数据应用于发送的对象（请参阅vision.image或数据块API，为什么这很重要）。

train_dl，valid_dl和可选的test_dl将包装在DeviceDataLoader中。

### Factory method
<b>create</b>

`create(train_ds:Dataset, valid_ds:Dataset, test_ds:Optional[Dataset]=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, num_workers:int=4, dl_tfms:Optional[Collection[Callable]]=None, device:device=None, collate_fn:Callable='data_collate', no_check:bool=False, **dl_kwargs) → DataBunch`

从train_ds，valid_ds和test_ds创建一个批处理大小为bs的DataBunch。 将** dl_kwargs传递给DataLoader（）

num_workers是要使用的CPU数，tfms，device和collate_fn是传递给init方法的。

`警告：您可以在这里传递常规的pytorch数据集，但是它们需要比基本数据库更多的属性才能使用库。 请参阅下面的更多细节。`

### Visualization
<b>show_batch</b>

`show_batch(rows:int=5, ds_type:DatasetType=<DatasetType.Train: 1>, reverse:bool=False, **kwargs)`

以ds_type显示一批数据在`rows`行中

### Grabbing some data
<b>dl</b>

`dl(ds_type:DatasetType=<DatasetType.Valid: 2>) → DeviceDataLoader`

返回适当的数据集以进行验证，培训或测试（ds_type）。

<b>one_batch</b>

`one_batch(ds_type:DatasetType=<DatasetType.Train: 1>, detach:bool=True, denorm:bool=True, cpu:bool=True) → Collection[Tensor]`

Get one batch from the data loader of ds_type. Optionally detach and denorm.

<b>one_item</b>

`one_item(item, detach:bool=False, denorm:bool=False, cpu:bool=False)`

从ds_type的数据加载器中获取一个批处理。 可选择分离和变形。

<b>sanity_check</b>
`sanity_check()`

检查训练集中的基础数据是否可以正确加载。

### Load and save
您可以使用此方法保存DataBunch对象以供将来使用。

<b>save</b>

`save(file:PathLikeOrBinaryStream='data_save.pkl')`

将DataBunch保存在self.path /文件中。 文件可以是文件（文件或缓冲区）

<b>load_data</b>

`load_data(path:PathOrStr, file:PathLikeOrBinaryStream='data_save.pkl', bs:int=64, val_bs:int=None, num_workers:int=4, dl_tfms:Optional[Collection[Callable]]=None, device:device=None, collate_fn:Callable='data_collate', no_check:bool=False, **kwargs) → DataBunch`

从路径/文件加载已保存的DataBunch。 文件可以是文件（文件或缓冲区）

`重要提示：您创建第一个`DataBunch`时传递的参数未保存，因此如果您不想使用默认值，则应将其传递给此处。`

这是为了让您轻松创建具有不同批量大小的新DataBunch。 您还需要重新应用您在原始DataBunch上执行的任何normalization（在视觉中）。

### Empty DataBunch for inference

<b>export</b>

`export(file:PathLikeOrBinaryStream='export.pkl')`

在self.path / file中导出self的最小状态以进行推理。 文件可以是文件（文件或缓冲区）

<b>load_empty</b>

`load_empty(path, fname:str='export.pkl')`

使用可选的tfms从path / fname中的导出文件加载空DataBunch。

此方法应该用于在推理中创建DataBunch，请参阅相应的教程。

<b>add_test</b>

`add_test(items:Iterator[T_co], label:Any=None)`

将项目添加为测试集。 传递标签，否则用EmptyLabel标记它们。

### Dataloader transforms

<b>add_tfm</b>

`add_tfm(tfm:Callable)`

向所有数据加载器添加转换。



## 在fastai中使用自定义数据集
如果要在fastai中使用pytorch数据集，则可能需要实现更多属性/方法（如果要使用库的完整功能）。 如果你只是添加一个属性，一些函数可以很容易地与你的pytorch数据集一起使用，对于其他函数，最好的方法是按照本教程创建你自己的ItemList。 这是库期望的完整列表。

### 基本
首先，您显然需要实现方法__len__和__getitem__，如pytorch文档所示。 然后最需要的东西是：

* c属性：它在大多数直接创建`Learner`的函数中使用（tabular_learner，text_classifier_learner，unet_learner，cnn_learner），并表示模型最后一层的输出数量（如果适用，还包括类的数量）。
* classes属性：它由ClassificationInterpretation和collab_learner使用（最好使用CollabDataBunch.from_df而不是pytorch数据集），并表示数据中出现的唯一标记。
* 也许是一个loss_func属性：学习者将把它用作默认的丢失函数，所以如果你知道你的自定义数据集需要一个特定的损失，你可以把它。

具有numpy数组和二进制标签的图像玩具示例:
```python
class ArrayDataset(Dataset):
    "Sample numpy array dataset"
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.c = 2 # binary label
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]
```

Code sample:
```python
train_x = np.random.rand(10, 3, 3) # 10 images (3x3)
train_y = np.random.rand(10, 1).round() # binary label

valid_x = np.random.rand(10, 3, 3)
valid_y = np.random.rand(10, 1).round()

train_ds, valid_ds = ArrayDataset(train_x, train_y), ArrayDataset(valid_x, valid_y)
data = DataBunch.create(train_ds, valid_ds, bs=2, num_workers=1)
data.one_batch()
```

Output:
```output
(tensor([[[0.8053, 0.5914, 0.5369],
          [0.6880, 0.4680, 0.5457],
          [0.0051, 0.2096, 0.3469]],
 
         [[0.5170, 0.2542, 0.9869],
          [0.0176, 0.5049, 0.4417],
          [0.3495, 0.7276, 0.5426]]], dtype=torch.float64), tensor([[0.],
         [0.]], dtype=torch.float64))
```

### 对于特定的应用程序
在文本中，您的数据集需要具有应该是Vocab实例的词汇属性。 构建模型时，text_classifier_learner和language_model_learner使用它。

在表格中，您的数据集将需要具有cont_names属性（对于连续变量的名称）和get_emb_szs方法，该方法返回元组列表（n_classes，emb_sz），表示对于每个分类变量，不同代码的数量（don'） 忘记为nan添加1）和相应的嵌入大小。 这两个与tabular_learner的c属性一起使用。

### 这些功能真的不起作用
要使这些最后的函数有效，您确实需要使用数据块API，并且可能编写自己的自定义ItemList。

DataBunch.show_batch（需要.x.reconstruct，.y.reconstruct和.x.show_xys）
Learner.predict（需要x.set_item，.y.analyze_pred，.y.reconstruct和.x.reconstruct）
Learner.show_results（需要x.reconstruct，y.analyze_pred，y.reconstruct和x.show_xyzs）
DataBunch.set_item（需要x.set_item）
Learner.backward（使用DataBunch.set_item）
DataBunch.export（需要导出）

## class DeviceDataLoader
`DeviceDataLoader(dl:DataLoader, device:device, tfms:List[Callable]=None, collate_fn:Callable='data_collate')`

将DataLoader绑定到torch.device。

在应用可选的tfms列表后，将dl批次放在设备上。 collate_fn将替换dl中的一个。 DataBunch的所有数据加载器都属于这种类型。

### Factory method
<b>create</b>

`create(dataset:Dataset, bs:int=64, shuffle:bool=False, device:device=device(type='cuda'), tfms:Collection[Callable]=None, num_workers:int=4, collate_fn:Callable='data_collate', **kwargs:Any)`

使用bs和shuffle从数据集创建DeviceDataLoader：使用num_workers进行处理。

给定的collate_fn将用于将样本放在一个批处理中（默认情况下，它会抓取它们的数据属性）。 shuffle表示如果该标志设置为True，则数据加载器将随机采样，否则采用正确的顺序。 tfms传递给init方法。 所有kwargs都传递给pytorch DataLoader类初始化。

### Methods
<b>add_tfm</b>

`add_tfm(tfm:Callable)`

将tfm添加到self.tfms。

<b>remove_tfm</b>

`remove_tfm(tfm:Callable)`

从self.tfms中删除tfm。

<b>new</b>

`new(**kwargs)`

使用kwargs替换当前值创建self的新副本。

<b>proc_batch</b>

`proc_batch(b:Tensor) → Tensor`

处理TensorImage的批处理b。

## DatasetType
Enum = [Train, Valid, Test, Single, Fix]

内部枚举器，用于命名培训，验证和测试数据集/数据加载器。